In [1]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(file_path="knowledge_base.json", jq_schema=".[].content", text_content=True)
documents = loader.load()
documents

[Document(metadata={'source': '/Users/surajpandey/Documents/Python/agentic-ai-wrk/RAGS/knowledge_base.json', 'seq_num': 1}, page_content="Transformer models were introduced in the paper 'Attention Is All You Need' by Vaswani et al. in 2017. The architecture relies on self-attention mechanisms rather than recurrent or convolutional neural networks. This design allows for more parallelization during training and better handling of long-range dependencies in text."),
 Document(metadata={'source': '/Users/surajpandey/Documents/Python/agentic-ai-wrk/RAGS/knowledge_base.json', 'seq_num': 2}, page_content='BERT (Bidirectional Encoder Representations from Transformers) was developed by Google AI Language team in 2018. It is pre-trained using masked language modeling and next sentence prediction tasks. BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.'),
 Document(metadata={'source': '/Users/surajpandey/Documen

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=150,
    chunk_overlap=20,
)

document = """# Introduction to RAG
Retrieval-Augmented Generation (RAG) combines retrieval systems with
generative AI models.
It helps address hallucinations by grounding responses in retrieved
information.
## Key Components
RAG consists of several components:
1. Document processing
2. Vector embedding
3. Retrieval
4. Augmentation
5. Generation
### Document Processing
This step involves loading and chunking documents appropriately.
"""

In [5]:
chunks = text_splitter.split_text(document)

In [6]:
chunks

['# Introduction to RAG\nRetrieval-Augmented Generation (RAG) combines retrieval systems with\ngenerative AI models.',
 'It helps address hallucinations by grounding responses in retrieved\ninformation.\n## Key Components\nRAG consists of several components:',
 '1. Document processing\n2. Vector embedding\n3. Retrieval\n4. Augmentation\n5. Generation\n### Document Processing',
 'This step involves loading and chunking documents appropriately.']

In [7]:
# Semantical chunking - chunk based on meaning of content

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
text_splitter = SemanticChunker(
    embeddings=embeddings,
    add_start_index=True,
)

chunks = text_splitter.split_text(document)

In [8]:
chunks

['# Introduction to RAG\nRetrieval-Augmented Generation (RAG) combines retrieval systems with\ngenerative AI models. It helps address hallucinations by grounding responses in retrieved\ninformation. ## Key Components\nRAG consists of several components:\n1. Document processing\n2. Vector embedding\n3. Retrieval\n4.',
 'Augmentation\n5. Generation\n### Document Processing\nThis step involves loading and chunking documents appropriately. ']

In [11]:
# hybrid retriever

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
# Initialize the embeddings model
embeddings_model = OpenAIEmbeddings()

vector_store = FAISS.from_documents(documents=documents, embedding=embeddings_model)

# Setup semantic retriever
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Setup lexical retriever
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 5

# combine retrievers
hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3],  # Adjust weights as needed
)

results = hybrid_retriever.get_relevant_documents("What is RAG?")

results

/var/folders/cn/rj2hj5wn5cb0cxqfzy04f8c80000gn/T/ipykernel_40627/3475046693.py:25: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = hybrid_retriever.get_relevant_documents("What is RAG?")


[Document(id='41ae0a3d-e7af-47c1-89b4-3195c45888c6', metadata={'source': '/Users/surajpandey/Documents/Python/agentic-ai-wrk/RAGS/knowledge_base.json', 'seq_num': 4}, page_content='Retrieval-Augmented Generation (RAG) combines a retrieval system with a text generator. The retriever fetches relevant documents from a knowledge base, and these documents are then provided as context to the generator. RAG models can be fine-tuned end-to-end and leverage large pre-trained models like BART or T5 for generation. This approach helps ground the generated text in factual information.'),
 Document(id='b73170e3-ee75-44c5-80c1-bc256690e28b', metadata={'source': '/Users/surajpandey/Documents/Python/agentic-ai-wrk/RAGS/knowledge_base.json', 'seq_num': 2}, page_content='BERT (Bidirectional Encoder Representations from Transformers) was developed by Google AI Language team in 2018. It is pre-trained using masked language modeling and next sentence prediction tasks. BERT is designed to pre-train deep bid